In [1]:
import json

In [5]:
dataset = []
with open("top_ai_subset.txt", "r") as f:
    line = f.readline()
    while line:
        sl = json.loads(line)
        dataset.append(sl)
        line = f.readline()

In [41]:
def abstract_reconstruct(abstract): 
    new_abstract = {}
    for key, value in abstract.items():
        for va in value:
            new_abstract[va] = key
    new_abstract = sorted(new_abstract.items(), key = lambda item:item[0])
    abstract_text = ""
    for (index, word) in new_abstract:
        abstract_text += " " + word
    return abstract_text[1:]

In [27]:
_all_feature = set()
for paper in dataset:
    _all_feature = _all_feature.union(set(paper.keys()))
_remove_col = set(['doc_type', 'doi', 'fos', 'issue', 'page_end', 'page_start','volume','publisher','authors'])
_all_needed_feature = _all_feature - _remove_col

In [28]:
_all_needed_feature

{'id',
 'indexed_abstract',
 'n_citation',
 'references',
 'title',
 'venue',
 'year'}

In [25]:
import csv

In [35]:
dataset[0]

{'id': '100638832',
 'title': 'Learning English Light Verb Constructions: Contextual or Statistical',
 'authors': [{'name': 'Yuancheng Tu',
   'id': '2161005799',
   'org': 'University of Illinois#TAB#'},
  {'name': 'Dan Roth',
   'id': '2122007671',
   'org': 'University of Illinois#TAB#'}],
 'venue': {'raw': 'meeting of the association for computational linguistics',
  'id': '1188739475'},
 'year': 2011,
 'n_citation': 28,
 'page_start': '31',
 'page_end': '39',
 'doc_type': 'Conference',
 'publisher': 'Association for Computational Linguistics',
 'volume': '',
 'issue': '',
 'references': ['40123913',
  '85238235',
  '1498763386',
  '1579558000',
  '1593045043',
  '1976632482',
  '2123215530',
  '2125712079',
  '2153635508'],
 'indexed_abstract': {'IndexLength': 138,
  'InvertedIndex': {'In': [0, 42],
   'this': [1, 101, 119],
   'paper,': [2],
   'we': [3, 44, 103],
   'investigate': [4],
   'a': [5, 26, 106, 128],
   'supervised': [6],
   'machine': [7],
   'learning': [8, 12],
  

In [86]:
def Classify_GT(venue):
    if venue in ['international joint conference on artificial intelligence']:
        return "AI"
    if venue in ['computer vision and pattern recognition', 'international conference on computer vision', 'european conference on computer vision']:
        return "CV"
    if venue in ['international conference on machine learning', 'knowledge discovery and data mining', 'neural information processing systems']:
        return "ML"
    if venue in ['empirical methods in natural language processing', 'north american chapter of the association for computational linguistics', 'meeting of the association for computational linguistics']:
        return "NLP"
    if venue in ['international acm sigir conference on research and development in information retrieval', 'international world wide web conferences']:
        return "WI"

In [87]:
# paper_id = dataset[0]["id"]
def paper_extract(paper):
    paper_id = paper["id"]
    paper_abstract_len = paper["indexed_abstract"]['IndexLength']
    paper_abstract = abstract_reconstruct(paper["indexed_abstract"]['InvertedIndex'])
    paper_n_citation = paper["n_citation"]
    paper_title = paper["title"]
    paper_venue = paper["venue"]['raw']
    paper_year = paper["year"]
    paper_refer = paper["references"]
    label = Classify_GT(paper_venue)
    return [paper_id, paper_abstract_len, paper_abstract, paper_n_citation, paper_title, paper_venue, paper_year, label]

In [89]:
data_to_be_write = []
for i in range(len(dataset)):
    try:
        data_to_be_write.append(paper_extract(dataset[i]))
    except:
        pass

In [88]:
header = ["id", "abstract_len", "abstract", "num_cite", "title", "venue", "year", "label"]

In [90]:
csv_file = open("paper_data_csv.csv", "w", encoding='utf-8', newline='')
csv_writer = csv.writer(csv_file,dialect='excel')
csv_writer.writerow(header)
csv_writer.writerows(data_to_be_write)
csv_file.close()

In [72]:
import pandas as pd

In [151]:
dataset_df = pd.read_csv("paper_data_csv.csv")

In [152]:
dataset_df

,id,abstract_len,abstract,num_cite,title,venue,year,label
0,100638832,138,"In this paper, we investigate a supervised mac...",28,Learning English Light Verb Constructions: Con...,meeting of the association for computational l...,2011,NLP
1,101796631,105,Functional chunks are defined as a series of n...,3,A SVM-Based Model for Chinese Functional Chunk...,meeting of the association for computational l...,2006,NLP
2,1019104620,104,"We present a novel compositional, generative m...",5,A Generative Model of Vector Space Semantics,meeting of the association for computational l...,2013,NLP
3,102083554,193,This paper presents the participation of the U...,6,UBA: Using Automatic Translation and Wikipedia...,meeting of the association for computational l...,2010,NLP
4,102440339,66,In this paper I present a novel Machine Learni...,21,Learning Morphology with Pair Hidden Markov Mo...,meeting of the association for computational l...,2001,NLP
...,...,...,...,...,...,...,...,...
66205,98102903,82,Most existing learning to rank based recommend...,11,SoRank: incorporating social information into ...,international world wide web conferences,2014,WI
66206,98239058,130,"In Twitter, users can annotate tweets with has...",2,Learning to annotate tweets with crowd wisdom,international world wide web conferences,2013,WI
66207,99223961,121,This work presents a novel method for the gene...,2,Spanish knowledge base generation for polarity...,international world wide web conferences,2013,WI
66208,99556955,86,We consider the problem of improving the perfo...,0,Improving Web Site's Accessibility.,international world wide web conferences,2001,WI


In [98]:
from sklearn.neural_network import MLPClassifier
import sklearn

In [153]:
dataset_df = sklearn.utils.shuffle(dataset_df,random_state=671)

In [154]:
train_num = int(0.8*len(dataset_df))
train_df = dataset_df[:train_num]
test_df = dataset_df[train_num:]

In [155]:
x = train_df.drop(columns=["id","abstract", "title", "venue", "label"]).values

In [156]:
y = train_df.label.values

In [157]:
mlp = MLPClassifier().fit(x, y)

In [158]:
x_test = test_df.drop(columns=["id","abstract", "title", "venue", "label"]).values
y_test = test_df.label.values

In [159]:
count = 0
ourpred = []
for i in range(len(x_test)):
    if mlp.predict(x_test[i].reshape(1, -1)) == y_test[i]:
        count += 1
        print("{}/{}".format(count,i+1))
    else:
        print("our prediction: ", mlp.predict(x_test[i].reshape(1, -1)), "actrually prediction: ", y_test[i])
    ourpred.append(mlp.predict(x_test[i].reshape(1, -1)))

our prediction:  ['NLP'] actrually prediction:  AI
our prediction:  ['NLP'] actrually prediction:  AI
our prediction:  ['NLP'] actrually prediction:  AI
1/4
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['CV'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  ML
2/8
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['CV'] actrually prediction:  WI
our prediction:  ['CV'] actrually prediction:  WI
3/12
our prediction:  ['NLP'] actrually prediction:  WI
4/14
our prediction:  ['CV'] actrually prediction:  ML
5/16
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
6/20
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
7/26
8/2

our prediction:  ['CV'] actrually prediction:  ML
67/244
our prediction:  ['NLP'] actrually prediction:  CV
68/246
69/247
70/248
71/249
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  CV
72/253
73/254
74/255
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['ML'] actrually prediction:  WI
our prediction:  ['CV'] actrually prediction:  AI
our prediction:  ['NLP'] actrually prediction:  CV
75/261
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['CV'] actrually prediction:  WI
76/267
our prediction:  ['CV'] actrually prediction:  WI
77/269
our prediction:  ['NLP'] actrually prediction:  ML
78/271
79/272
our prediction:  ['CV'] actrually prediction:  ML

our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  AI
our prediction:  ['CV'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  ML
153/489
154/490
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  AI
155/493
our prediction:  ['CV'] actrually prediction:  AI
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['CV'] actrually prediction:  AI
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  WI
156/499
157/500
158/501
159/502
our prediction:  ['NLP'] actrually prediction:  WI
160/504
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
161/507
our prediction:  ['CV'] actrually prediction:  NLP
162/509
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
our p

our prediction:  ['NLP'] actrually prediction:  ML
227/726
228/727
229/728
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['CV'] actrually prediction:  ML
230/731
231/732
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  CV
232/735
our prediction:  ['NLP'] actrually prediction:  CV
233/737
234/738
our prediction:  ['NLP'] actrually prediction:  AI
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['CV'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  ML
235/743
236/744
our prediction:  ['NLP'] actrually prediction:  AI
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
237/749
238/750
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually predi

our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['CV'] actrually prediction:  NLP
our prediction:  ['NLP'] actrually prediction:  ML
306/970
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  AI
307/973
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  AI
308/976
309/977
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  AI
our prediction:  ['CV'] actrually prediction:  NLP
our prediction:  ['WI'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
310/983
311/984
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  WI
312/989
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actr

...

our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  CV
3770/12061
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
3771/12067
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
3772/12078
our prediction:  ['NLP'] actrually prediction:  CV
3773/12080
3774/12081
our prediction:  ['NLP'] actru

our prediction:  ['NLP'] actrually prediction:  CV
3836/12294
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
3837/12297
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  AI
3838/12300
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['CV'] actrually prediction:  WI
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  AI
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
3839/12309
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['CV'] actrually prediction:  NLP
3840/12315
our prediction:  ['NLP'] actrua

3904/12542
3905/12543
our prediction:  ['CV'] actrually prediction:  AI
our prediction:  ['CV'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  ML
3906/12547
our prediction:  ['NLP'] actrually prediction:  CV
3907/12549
3908/12550
our prediction:  ['NLP'] actrually prediction:  CV
3909/12552
our prediction:  ['CV'] actrually prediction:  NLP
our prediction:  ['NLP'] actrually prediction:  CV
3910/12555
3911/12556
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['CV'] actrually prediction:  ML
3912/12560
our prediction:  ['CV'] actrually prediction:  ML
3913/12562
3914/12563
3915/12564
our prediction:  ['CV'] actrually prediction:  ML
3916/12566
our prediction:  ['CV'] actrually prediction:  ML
3917/12568
our prediction:  ['CV'] actrually prediction:  AI
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
3918/12572
our prediction:  ['NLP'] a

our prediction:  ['NLP'] actrually prediction:  AI
3983/12781
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  ML
3984/12789
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['CV'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['CV'] actrually prediction:  AI
3985/12798
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  CV
3

our prediction:  ['CV'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  CV
4057/13040
our prediction:  ['CV'] actrually prediction:  AI
our prediction:  ['CV'] actrually prediction:  NLP
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  CV
4058/13048
our prediction:  ['CV'] actrually prediction:  WI
our prediction:  ['NLP'] actrually prediction:  AI
4059/13051
our prediction:  ['NLP'] actrually prediction:  AI
4060/13053
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['NLP'] actrually prediction:  CV
our prediction:  ['CV'] actrually prediction:  ML
our prediction:  ['NLP'] actrually prediction:  ML
our prediction:  ['NLP'] actrually predict

In [160]:
from sklearn.metrics import f1_score
f1_score(y_test, ourpred, average='weighted')

0.22152857125070838

In [122]:
import nltk

In [163]:
total_words = [0,0,0,0,0]
class_count = [0,0,0,0,0]
dictionary = {}
count = 0
for i in range(len(train_df)):
    label = train_df.label.values[i]
    abstract = train_df.abstract.values[i]
    count += 1
    if label == "AI":
        class_count[0] += 1
    if label == "CV":
        class_count[1] += 1
    if label == "ML":
        class_count[2] += 1
    if label == "NLP":
        class_count[3] += 1
    if label == "WI":
        class_count[4] += 1
    res_list = nltk.word_tokenize(abstract)
    for word in res_list:
        if not word in dictionary:
            dictionary[word] = {"label_count":[1,1,1,1,1]}
        if label == "AI":
            dictionary[word]["label_count"][0] += 1
            total_words[0] += 1
        if label == "CV":
            dictionary[word]["label_count"][1] += 1
            total_words[1] += 1
        if label == "ML":
            dictionary[word]["label_count"][2] += 1
            total_words[2] += 1
        if label == "NLP":
            dictionary[word]["label_count"][3] += 1
            total_words[3] += 1
        if label == "WI":
            dictionary[word]["label_count"][4] += 1
            total_words[4] += 1
    if count % 1000 == 1:
        print(count)

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001
20001
21001
22001
23001
24001
25001
26001
27001
28001
29001
30001
31001
32001
33001
34001
35001
36001
37001
38001
39001
40001
41001
42001
43001
44001
45001
46001
47001
48001
49001
50001
51001
52001


In [164]:
fw = open("nb.txt", "w")
jsObj = json.dumps(dictionary)
fw.write(jsObj)
fw.close()

In [166]:
for key, value in dictionary.items():
    dictionary[key]["label_count"][0] = dictionary[key]["label_count"][0]/total_words[0]
    dictionary[key]["label_count"][1] = dictionary[key]["label_count"][1]/total_words[1]
    dictionary[key]["label_count"][2] = dictionary[key]["label_count"][2]/total_words[2]
    dictionary[key]["label_count"][3] = dictionary[key]["label_count"][3]/total_words[3]
    dictionary[key]["label_count"][4] = dictionary[key]["label_count"][4]/total_words[4]

In [171]:
from math import log
sumval = 0
for i in range(len(class_count)):
    sumval += class_count[i]
for i in range(len(class_count)):
    class_count[i] = class_count[i]/sumval
print(class_count)
correct = 0
label_count = [0,0,0,0,0]
for i in range(len(test_df)):
    label = test_df.label.values[i]
    abstract = test_df.abstract.values[i]
    res_list = nltk.word_tokenize(abstract)
    p_r = [0,0,0,0,0]
    for word in res_list:
        if not word in dictionary:
            continue
        p_r[0] += log(dictionary[word]["label_count"][0])
        p_r[1] += log(dictionary[word]["label_count"][1])
        p_r[2] += log(dictionary[word]["label_count"][2])
        p_r[3] += log(dictionary[word]["label_count"][3])
        p_r[4] += log(dictionary[word]["label_count"][4])
    p_r[0] += log(class_count[0])
    p_r[1] += log(class_count[1])
    p_r[2] += log(class_count[2])
    p_r[3] += log(class_count[3])
    p_r[4] += log(class_count[4])
    maxval = -float("inf")
    maxindex = 0
    # print(p_r)
    for i in range(5):
        if p_r[i] > maxval:
            maxval = p_r[i]
            maxindex = i
    if label == "AI":
        label_count[0] += 1
        if maxindex == 0:
            correct += 1
    if label == "CV":
        label_count[1] += 1
        if maxindex == 1:
            correct += 1
    if label == "ML":
        label_count[2] += 1
        if maxindex == 2:
            correct += 1
    if label == "NLP":
        label_count[3] += 1
        if maxindex == 3:
            correct += 1
    if label == "WI":
        label_count[4] += 1
        if maxindex == 4:
            correct += 1

[0.09977722398429241, 0.3278583295574687, 0.2583824195740825, 0.16946080652469417, 0.14452122035946233]


In [172]:
correct/len(test_df)

0.7988219302220209